# Pandas

## Introduction

- library for Data Analysis and Manipulation

**Why Pandas?**

- provides ability to work with Tabular data
  - `Tabular Data` : data that is organized into tables having rows and cols

In [94]:
""" 
jupyter nbconvert --to markdown pandas.ipynb --output README.md

 """
import pandas as pd
import numpy as np

## `Series` objects

- A `Series` object is 1D array that can hold/store data.

### Creating a `Series`

In [9]:
l = ["C", "C++", "Python", "Javascript"]
s = pd.Series(l)
s

0             C
1           C++
2        Python
3    Javascript
dtype: object

### Similar to a 1D `ndarray`

`Series` objects behave much like one-dimensional NumPy `ndarray`s, and you can often pass them as parameters to NumPy functions:

In [3]:
import numpy as np

s = pd.Series([2,4,6,8])
np.exp(s)

0       7.389056
1      54.598150
2     403.428793
3    2980.957987
dtype: float64

Arithmetic operations on `Series` are also possible, and they apply *elementwise*, just like for `ndarray`s:

In [4]:
s + [1000,2000,3000,4000]

0    1002
1    2004
2    3006
3    4008
dtype: int64

Similar to NumPy, if you add a single number to a `Series`, that number is added to all items in the `Series`. This is called * broadcasting*:

In [5]:
s + 1000

0    1002
1    1004
2    1006
3    1008
dtype: int64

The same is true for all binary operations such as `*` or `/`, and even conditional operations:

In [6]:
s < 0

0    False
1    False
2    False
3    False
dtype: bool

### Indexing

In [28]:
s2 = pd.Series(l, index=["a", "b", "c", "d"])
s2

a             C
b           C++
c        Python
d    Javascript
dtype: object

You can then use the `Series` just like a `dict`:

In [11]:
s2["b"]

'C++'

You can still access the items by integer location, like in a regular array:

In [12]:
s2[1]

'C++'

To make it clear when you are accessing by label or by integer location, it is recommended to always use the `loc` attribute when accessing by label, and the `iloc` attribute when accessing by integer location:

In [13]:
s2.loc["b"]

'C++'

In [14]:
s2.iloc[1]

'C++'

#### Slicing a `Series` also slices the index labels:

In [15]:
s2.iloc[1:3]

b       C++
c    Python
dtype: object

This can lead to unexpected results when using the default numeric labels, so be careful:

In [16]:
surprise = pd.Series([1000, 1001, 1002, 1003])
surprise

0    1000
1    1001
2    1002
3    1003
dtype: int64

In [17]:
surprise_slice = surprise[2:]
surprise_slice

2    1002
3    1003
dtype: int64

Oh look! The first element has index label `2`. The element with index label `0` is absent from the slice:

In [18]:
try:
    surprise_slice[0]
except KeyError as e:
    print("Key error:", e)

Key error: 0


But remember that you can access elements by integer location using the `iloc` attribute. This illustrates another reason why it's always better to use `loc` and `iloc` to access `Series` objects:

In [19]:
surprise_slice.iloc[0]

1002

#### Init from `dict`

You can create a `Series` object from a `dict`. The keys will be used as index labels:

In [20]:
weights = {"a": 68, "b": 83, "c": 86, "d": 68}
s3 = pd.Series(weights)
s3

a    68
b    83
c    86
d    68
dtype: int64

You can control which elements you want to include in the `Series` and in what order by explicitly specifying the desired `index`:

In [24]:
s4 = pd.Series(weights, index = ["c", "a"])
s4

c    86
a    68
dtype: int64

### Automatic alignment

When an operation involves multiple `Series` objects, `pandas` automatically aligns items by matching index labels.

In [34]:
s2 = pd.Series([1,2,3], index=["a", "b", "c"])
s3 = pd.Series([10,20,40], index=["a", "b", "d"])

print(s2.keys())
print(s3.keys())

s2 + s3


Index(['a', 'b', 'c'], dtype='object')
Index(['a', 'b', 'd'], dtype='object')


a    11.0
b    22.0
c     NaN
d     NaN
dtype: float64

The resulting `Series` contains the union of index labels from `s2` and `s3`. Since `"d"` is missing from `s2` and `"c"` is missing from `s3`, these items have a `NaN` result value. (ie. Not-a-Number means *missing*).

Automatic alignment is very handy when working with data that may come from various sources with varying structure and missing items. But if you forget to set the **right index labels**, you can have surprising results:

In [36]:
s5 = pd.Series([1000,1000,1000,1000])
s2 + s5

a   NaN
b   NaN
c   NaN
0   NaN
1   NaN
2   NaN
3   NaN
dtype: float64

Pandas could not align the `Series`, since their labels do not match at all, hence the full `NaN` result.

### Init with a scalar

You can also initialize a `Series` object using a scalar and a list of index labels: all items will be set to the scalar.

In [37]:
meaning = pd.Series(42, ["a", "b", "c"])
meaning

a    42
b    42
c    42
dtype: int64

In [38]:
s6 = pd.Series([83, 68], index=["a", "b"], name="weights")
s6

a    83
b    68
Name: weights, dtype: int64

## `DataFrame` objects

A `DataFrame` is a table. It contains an array of individual entries, each of which has a certain value. Each entry corresponds to a row (or record) and a column.

- A DataFrame object represents a 2d labelled array, with cell values, column names and row index labels
- You can see `DataFrame`s as dictionaries of `Series`.



<div align="center">
<img src="img/anatomy.png" alt="anatomy.jpg" width="1000px">
</div>

## Creating a `DataFrame`

### From Numpy Array

In [51]:
arr = np.random.randint(10,100,size=(6,4))
arr

array([[30, 27, 82, 14],
       [94, 66, 75, 56],
       [53, 19, 72, 20],
       [32, 91, 10, 14],
       [88, 65, 70, 49],
       [31, 57, 27, 95]])

In [53]:
df = pd.DataFrame(data=arr)
df

,0,1,2,3
0,30,27,82,14
1,94,66,75,56
2,53,19,72,20
3,32,91,10,14
4,88,65,70,49
5,31,57,27,95


In [91]:
arr = np.random.randint(10, 100, size=(6, 4))
df = pd.DataFrame(data=arr)
df.columns = ["a", "b", "c", "d"]
df.index = "p q r s t u".split()
df


,a,b,c,d
p,19,51,72,11
q,92,26,88,15
r,68,10,90,14
s,46,61,37,41
t,12,78,48,93
u,29,28,17,40


In [93]:
np.random.seed(5)
arr=np.random.randint(100, size=(5, 5))
df = pd.DataFrame(arr, 
				columns=list("ABCDE"),
                index=["R" + str(i) for i in range(5)])
df

,A,B,C,D,E
R0,99,78,61,16,73
R1,8,62,27,30,80
R2,7,76,15,53,80
R3,27,44,77,75,65
R4,47,30,84,86,18


For more see [`DataFrame(columns=[],index=[])` constructor](#dataframecolumnsindex-constructor)


### Using `dictionary` of `List`, `pd.Series`, `np.Array`:

The syntax for declaring a new one is a `dictionary` whose `keys` are the `column` names (`col1`, `col2`, `col3` ..in this example), and whose **values are a `list` of entries**. This is the standard way of constructing a new DataFrame, and the one you are most likely to encounter.

In [96]:
df = pd.DataFrame({
	'col1': [10, 3, 2, 4],
	'col2': [111, 112, 113, 114],
	'col3': [90, 80, 70, 60]
})
df


,col1,col2,col3
0,10,111,90
1,3,112,80
2,2,113,70
3,4,114,60


The dictionary-list constructor assigns values to the column labels, **but just uses an ascending count from 0 (0, 1, 2, 3, ...) for the row labels**. Sometimes this is OK, but oftentimes we will want to assign these labels ourselves.

The list of row labels used in a DataFrame is known as an `Index`. We can assign values to it by using an index parameter in our constructor:

In [95]:
df = pd.DataFrame({
	'col1': [10, 3, 2, 4],
	'col2': [111, 112, 113, 114],
	'col3': [90, 80, 70, 60]
},index=["row1", "row2", "row3", "row4"])
df


,col1,col2,col3
row1,10,111,90
row2,3,112,80
row3,2,113,70
row4,4,114,60


In [6]:
user_data = {
	"MarksA": np.random.randint(1,100,5),
	"MarksB": np.random.randint(50,100,5),
	"MarksC": np.random.randint(1,100,5)
}
df = pd.DataFrame(user_data)
df.head(n=3)

,MarksA,MarksB,MarksC
0,50,55,97
1,30,92,30
2,66,97,37


In [160]:
df = pd.DataFrame({
	"id": np.arange(10),
	'b': np.random.normal(size=10),
	"c": pd.Series(np.random.choice(["cat", 'dog', "hippo"], replace=True, size=10))
})
df.head()


,id,b,c
0,0,-0.736681,dog
1,1,-0.284158,cat
2,2,0.213199,cat
3,3,-2.400537,cat
4,4,-0.112093,hippo


In [28]:
people_dict = {
    "weight": pd.Series([68, 83, 112], index=["alice", "bob", "charles"]),
    "birthyear": pd.Series([1984, 1985, 1992], index=["bob", "alice", "charles"], name="year"),
    "children": pd.Series([0, 3], index=["charles", "bob"]),
    "hobby": pd.Series(["Biking", "Dancing"], index=["alice", "bob"]),
}
people = pd.DataFrame(people_dict)
people

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


A few things to note:
* the `Series` were automatically aligned based on their index,
* missing values are represented as `NaN`,
* `Series` names are ignored (the name `"year"` was dropped),
* `DataFrame`s are displayed nicely in Jupyter notebooks, woohoo!

You can access columns pretty much as you would expect. They are returned as `Series` objects:

In [51]:
people["birthyear"]

alice      1985
bob        1984
charles    1992
Name: birthyear, dtype: int64

You can also get multiple columns at once:

In [52]:
people[["birthyear", "hobby"]]

,birthyear,hobby
alice,1985,Biking
bob,1984,Dancing
charles,1992,NaN


For more see [Indexing, Masking, Query](#indexing-masking-query)

It is also possible to create a `DataFrame` with a dictionary (or list) of dictionaries (or list):

In [61]:
people = pd.DataFrame({
    "birthyear": {"alice": 1985, "bob": 1984, "charles": 1992},
    "hobby": {"alice": "Biking", "bob": "Dancing"},
    "weight": {"alice": 68, "bob": 83, "charles": 112},
    "children": {"bob": 3, "charles": 0}
})
people


,birthyear,hobby,weight,children
alice,1985,Biking,68,NaN
bob,1984,Dancing,83,3.0
charles,1992,NaN,112,0.0


### `DataFrame(columns=[],index=[])` constructor

If you pass a list of columns and/or index row labels to the `DataFrame` constructor, it will guarantee that these columns and/or rows will exist, in that order, and no other column/row will exist. For example:

In [6]:
d2 = pd.DataFrame(
        people_dict,
        columns=["birthyear", "weight", "height"],
        index=["bob", "alice", "eugene"]
     )
d2

,birthyear,weight,height
bob,1984.0,83.0,NaN
alice,1985.0,68.0,NaN
eugene,NaN,NaN,NaN


Another convenient way to create a `DataFrame` is to pass all the values to the constructor as an `ndarray`, or a list of lists, and specify the column names and row index labels separately:

In [10]:
values = [
            [1985, np.nan, "Biking",   68],
            [1984, 3,      "Dancing",  83],
            [1992, 0,      np.nan,    112]
         ]
d3 = pd.DataFrame(
        values,
        columns=["birthyear", "children", "hobby", "weight"],
        index=["alice", "bob", "charles"]
     )
d3

,birthyear,children,hobby,weight
alice,1985,NaN,Biking,68
bob,1984,3.0,Dancing,83
charles,1992,0.0,NaN,112


To specify missing values, you can either use `np.nan` or NumPy's masked arrays:

In [55]:
masked_array = np.ma.asarray(values, dtype=np.object)
masked_array[(0, 2), (1, 2)] = np.ma.masked
d3 = pd.DataFrame(
        masked_array,
        columns=["birthyear", "children", "hobby", "weight"],
        index=["alice", "bob", "charles"]
     )
d3

,birthyear,children,hobby,weight
alice,1985,NaN,Biking,68
bob,1984,3,Dancing,83
charles,1992,0,NaN,112


Instead of an `ndarray`, you can also pass a `DataFrame` object:

In [11]:
d4 = pd.DataFrame(
         d3,
         columns=["hobby", "children"],
         index=["alice", "bob"]
     )
d4

,hobby,children
alice,Biking,NaN
bob,Dancing,3.0


## Indexing, Masking, Query

### 🚀Extracting Columns - Native accessors: `df[col]`, `df[[col1,col2,..]]`

In [7]:
np.random.seed(10)
arr = np.random.randint(10, 100, size=(6, 4))
df = pd.DataFrame(data=arr,columns=["a", "b", "c", "d"])
# df.columns = ["a", "b", "c", "d"]
df

,a,b,c,d
0,19,25,74,38
1,99,39,18,83
2,10,50,46,26
3,21,64,98,72
4,43,82,88,59
5,61,64,87,79


In Python, we can access the property of an object by accessing it as an attribute. A `book` object, for example, might have a `title` property, which we can access by calling` book.title`. `Columns` in a pandas DataFrame work in much the same way.

In [8]:
df.c
# If column name has spaces, this will not work

0    74
1    18
2    46
3    98
4    88
5    87
Name: c, dtype: int32

If we have a Python dictionary, we can access its values using the indexing (`[]`) operator. We can do the same with `columns` in a DataFrame:

In [9]:
df['c']

0    74
1    18
2    46
3    98
4    88
5    87
Name: c, dtype: int32

Indexing operator `[]` does have the advantage that it can handle `column` names with **reserved characters** in them (e.g. if we had a `country providence` column, `reviews.country providence` wouldn't work).

Doesn't a pandas Series look kind of like a fancy dictionary? It pretty much is, so it's no surprise that, to drill down to a **single specific value**, we need only use the indexing operator `[]` once more:

In [10]:
df['c'][0]

74

multiple columns can be extracted at once:

In [11]:
df[['b','c','a']]

,b,c,a
0,25,74,19
1,39,18,99
2,50,46,10
3,64,98,21
4,82,88,43
5,64,87,61


### 🚀Index-based selection - `iloc[row_indexer,col_indexer]`


In [87]:
np.random.seed(10)
arr = np.random.randint(10, 100, size=(6, 4))
df = pd.DataFrame(data=arr)
df.columns = ["a", "b", "c", "d"]
df.index = "p q r s t u".split()
df

,a,b,c,d
p,19,25,74,38
q,99,39,18,83
r,10,50,46,26
s,21,64,98,72
t,43,82,88,59
u,61,64,87,79


Pandas indexing works in one of two paradigms. The first is **index-based selection**: ***selecting data based on its numerical position in the data***. `iloc` follows this paradigm.

In [85]:
first_row = df.iloc[0]
first_row

a    19
b    25
c    74
d    38
Name: p, dtype: int32

Both `loc` and `iloc` are `row-first, column-second`. This is the opposite of what we do in native `Python`, which is `column-first, row-second`.

This means that it's marginally easier to retrieve `rows`, and marginally harder to get retrieve `columns`. To get a column with `iloc`, we can do the following:

In [30]:
df.iloc[:, 0] # all rows, first column

p    19
q    99
r    10
s    21
t    43
u    61
Name: a, dtype: int32

On its own, the `:` operator, which also comes from native Python, means `"everything"`. When combined with other selectors, however, it can be used to indicate a range of values. For example, to select the country column from just the first, second, and third row, we would do:

Or, to select just the second and third entries, we would do:

In [19]:
df.iloc[1:3, 0] # second and third row, first column

q    99
r    10
Name: a, dtype: int32

It's also possible to pass a list:

In [20]:
df.iloc[[0, 1, 2], 0] # first three rows, first column

p    19
q    99
r    10
Name: a, dtype: int32

In [21]:
df.iloc[:, 0:3] # all rows, first three columns

,a,b,c
p,19,25,74
q,99,39,18
r,10,50,46
s,21,64,98
t,43,82,88
u,61,64,87


In [23]:
df.iloc[:2,:3] # first two rows, first three columns

,a,b,c
p,19,25,74
q,99,39,18


In [25]:
df.iloc[:2, [1,3]] # first two rows, second and fourth columns

,b,d
p,25,38
q,39,83


In [31]:
df.iloc[1,3]

83

In [34]:
df.iloc[1:3][['a','b']]

,a,b
q,99,39
r,10,50


In [36]:
df.iloc[1:3,[df.columns.get_loc(v) for v in ['a','b']]]

,a,b
q,99,39
r,10,50


### 🚀Label-based selection - `loc[row_indexer,col_indexer]`

The second paradigm for attribute selection is the one followed by the `loc` operator: **label-based selection**. In this paradigm, it's the **data index value**, **not its position**, which matters.

In [38]:
x = df.loc["p"]
print(type(x))
x

<class 'pandas.core.series.Series'>


a    19
b    25
c    74
d    38
Name: p, dtype: int32

Accessing a single row with list of labels returns a `DataFrame` object:

In [39]:
x1= df.loc[["p"]]
print(type(x1))
x1

<class 'pandas.core.frame.DataFrame'>


,a,b,c,d
p,19,25,74,38


`iloc` is conceptually simpler than loc because it ignores the dataset's indices. When we use iloc we treat the dataset like a big matrix (a list of lists), one that we have to index into by position. `loc`, by contrast, uses the information in the indices to do its work. Since your dataset usually has meaningful indices, it's usually easier to do things using loc instead. 

In [41]:
df.loc[["p","u"]]

,a,b,c,d
p,19,25,74,38
u,61,64,87,79


In [43]:
df.loc["p","a"]

19

In [42]:
df.loc[["p","u"],["a"]]

,a
p,19
u,61


#### Choosing between loc and iloc

When choosing or transitioning between `loc` and `iloc`, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes.

iloc uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So 0:10 will select entries 0,...,9. loc, meanwhile, indexes inclusively. So 0:10 will select entries 0,...,10.

Why the change? Remember that `loc` can **index any stdlib type:** `strings`, for example. If we have a `DataFrame` with `index` values `Apples,...,Potatoes, ...`, and we want to select *"all the alphabetical fruit choices between Apples and Potatoes"*, then it's a lot more convenient to index `df.loc['Apples':'Potatoes']` than it is to index something like `df.loc['Apples', 'Potatoet']` (t coming after s in the alphabet).

This is particularly **confusing when the DataFrame index is a simple numerical list**, e.g. `0,...,1000`. In this case `df.iloc[0:1000]` will return `1000` entries, while `df.loc[0:1000]` return `1001` of them! To get `1000` elements using `loc`, you will need to go one lower and ask for `df.loc[0:999]`.

Otherwise, the semantics of using `loc` are the same as those for `iloc`.

cols = ['country', 'variety']
df = reviews.loc[:99, cols]

equivalent to:

cols_idx = [0, 11]
df = reviews.iloc[:100, cols_idx]

### 🚀🚀Masking - Boolean Indexing

In [75]:
np.random.seed(5)
df = pd.DataFrame(np.random.randint(100, size=(5, 5)), columns = list("ABCDE"), 
                  index = ["R" + str(i) for i in range(5)])
df

,A,B,C,D,E
R0,99,78,61,16,73
R1,8,62,27,30,80
R2,7,76,15,53,80
R3,27,44,77,75,65
R4,47,30,84,86,18


In [87]:
df > 50

,A,B,C,D,E
R0,True,True,True,False,True
R1,False,True,False,False,True
R2,False,True,False,True,True
R3,False,False,True,True,True
R4,False,False,True,True,False


Example Dataset: [wine-reviews-dataset](https://www.kaggle.com/zynicide/wine-reviews)

In [48]:
reviews = pd.read_csv('winemag-data-130k-v2-mod.csv',index_col=0)
reviews.head(n=2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


We can start by checking if each wine is Italian or not: `country == 'Italy'`:

In [50]:
reviews.country == 'Italy'

0      True
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98     True
99    False
Name: country, Length: 100, dtype: bool

This operation produced a Series of `True/False` booleans based on the `country` of each `record`. 

To select **All Rows** where `country == 'Italy'`:

In [54]:
res = reviews[reviews.country == 'Italy']
res.head(n=2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo


This result can then be used inside of `loc` to select the relevant data:

In [55]:
res = reviews.loc[reviews.country == 'Italy']
res.head(n=2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo


In [58]:
mask = reviews.country == 'Italy'
cols = ['country', 'points', 'taster_name']
res = reviews.loc[mask, cols]
res.head(n=2)

,country,points,taster_name
0,Italy,87,Kerin O’Keefe
6,Italy,87,Kerin O’Keefe


Suppose we'll buy any wine that's made in `Italy` **or** which is rated above average. For this we use a `pipe` (`|`). For `and` -> `&`:

In [60]:
res = reviews.loc[(reviews.country == 'Italy') | (reviews.points >= 90), cols]
res.head(n=2)

,country,points,taster_name
0,Italy,87,Kerin O’Keefe
6,Italy,87,Kerin O’Keefe


I'm an economical wine buyer. Which wine is the "best bargain"? Create a variable `bargain_wine` with the title of the wine with the highest points-to-price ratio in the dataset.

In [9]:
bargain_idx = (reviews.points / reviews.price).idxmax()
print(bargain_idx)
bargain_wine = reviews.loc[bargain_idx, 'title']
bargain_wine

42


'Henry Fessy 2012 Nouveau  (Beaujolais)'

#### `isin`

`isin` is lets you select data whose value `"is in"`**a list of values**. For example, here's how we can use it to select wines only from `Italy` or `France`:



In [65]:
res = reviews.loc[reviews.country.isin(['Italy', 'France']),cols]
res.head(n=3)


,country,points,taster_name
0,Italy,87,Kerin O’Keefe
6,Italy,87,Kerin O’Keefe
7,France,87,Roger Voss


Create a DataFrame `top_oceania_wines` containing all reviews with at least `95 points` (out of 100) for wines from `Italy` or `France`.

In [98]:
top_oceania_wines = reviews.loc[
    (reviews.country.isin(['Italy', 'France']))
    & (reviews.points >= 80)
	,cols
]
top_oceania_wines.head(n=3)


,country,points,taster_name
0,Italy,87,Kerin O’Keefe
6,Italy,87,Kerin O’Keefe
7,France,87,Roger Voss


####  `isnull`

The second is `isnull` (and its companion `notnull`). These methods let you highlight values which are (or are not) e`mpty (`NaN`). For example, to filter out wines lacking a price tag in the dataset, here's what we would do:


In [71]:
res =  reviews.loc[reviews.price.isnull(),['country','price']]
res.head(n=3)


,country,price
0,Italy,NaN
13,Italy,NaN
30,France,NaN


In [72]:
res.shape

(8, 2)

In [70]:
res =  reviews.loc[reviews.price.notnull(),['country','price']]
res.shape

(92, 2)

### Querying a `DataFrame`

The `query()` method lets you filter a `DataFrame` based on a query expression:

In [87]:
people.query("age > 30 and pets == 0")

,hobby,height,weight,age,over 30,pets,body_mass_index,overweight
bob,Dancing,181,83,34,True,0.0,25.335002,False


## Summary Functions and Maps

Example Dataset: [wine-reviews-dataset](https://www.kaggle.com/zynicide/wine-reviews)

In [19]:
reviews = pd.read_csv('winemag-data-130k-v2-mod.csv', index_col=0)
reviews.head(n=2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,77,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


### `shape` , `dtypes` , `info()`, `describe()`

In [5]:
reviews.shape


(100, 13)

In [6]:
reviews.dtypes


country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

In [7]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                100 non-null    object 
 1   description            100 non-null    object 
 2   designation            70 non-null     object 
 3   points                 100 non-null    int64  
 4   price                  92 non-null     float64
 5   province               100 non-null    object 
 6   region_1               88 non-null     object 
 7   region_2               37 non-null     object 
 8   taster_name            82 non-null     object 
 9   taster_twitter_handle  74 non-null     object 
 10  title                  100 non-null    object 
 11  variety                100 non-null    object 
 12  winery                 100 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 10.9+ KB


In [8]:
reviews.describe()


,points,price
count,100.000000,92.000000
mean,86.430000,26.271739
std,0.794616,18.320170
min,85.000000,9.000000
25%,86.000000,14.000000
50%,86.000000,20.000000
75%,87.000000,30.000000
max,88.000000,100.000000


In [9]:
reviews.describe().T


,count,mean,std,min,25%,50%,75%,max
points,100.0,86.430000,0.794616,85.0,86.0,86.0,87.0,88.0
price,92.0,26.271739,18.320170,9.0,14.0,20.0,30.0,100.0


This method generates a high-level summary of the attributes of the given `column`. It is type-aware, meaning that its output changes based on the data type of the input. The output above only makes sense for numerical data; for string data here's what we get:

In [12]:
reviews.taster_name.describe()
# reviews['taster_name'].describe()

count             82
unique            12
top       Roger Voss
freq              16
Name: taster_name, dtype: object

### `head` and `tail`

- `head`: prints the first 5 rows
- `tail`: prints the last 5 rows

In [13]:
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [17]:
reviews.head(2)
reviews.head(n=2)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [18]:
reviews.tail(n=2)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
98,Italy,"Forest floor, menthol, espresso, cranberry and...",Dono Riserva,88,30.0,Tuscany,Morellino di Scansano,NaN,Kerin O’Keefe,@kerinokeefe,Serpaia di Endrizzi 2010 Dono Riserva (Morell...,Sangiovese,Serpaia di Endrizzi
99,US,This blends 20% each of all five red-Bordeaux ...,Intreccio Library Selection,88,75.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Soquel Vineyards 2013 Intreccio Library Select...,Bordeaux-style Red Blend,Soquel Vineyards


### `columns`

In [19]:
reviews.columns


Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

### `unique` and `nunique`

The Pandas Unique technique identifies the unique values of a Pandas Series.

Count Not of Unique Values

In [29]:
reviews.nunique()


country                   10
description              100
designation               65
points                     4
price                     34
province                  30
region_1                  56
region_2                  10
taster_name               12
taster_twitter_handle     10
title                    100
variety                   44
winery                    91
dtype: int64

In [31]:
# reviews['country'].nunique()
reviews.country.nunique()

10

In [36]:
reviews['country'].unique()

array(['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria'], dtype=object)

### `value_counts()`

count occupance of each unique element

In [37]:
reviews['country'].value_counts()

US           43
Italy        24
France       14
Chile         5
Germany       4
Spain         3
Australia     2
Portugal      2
Argentina     2
Austria       1
Name: country, dtype: int64

In [39]:
reviews['country'].value_counts()['US']

43

### Maps

A `map` is a term, borrowed from mathematics, for a function that takes one set of values and "maps" them to another set of values. 

In data science we often have a need for **creating new representations from existing data**, *or* for **transforming data from one format to another**.

`Maps` are what handle this work, making them extremely important for getting your work done! There are two mapping methods that you will use often- `map()` and `apply()`.

map() is the first, and slightly simpler one. For example, suppose that we wanted to remean the scores the wines received to 0. We can do this as follows:


In [40]:
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)

0     0.57
1     0.57
2     0.57
3     0.57
4     0.57
      ... 
95    1.57
96    1.57
97    1.57
98    1.57
99    1.57
Name: points, Length: 100, dtype: float64

The function you pass to `map()` should **expect** a single value from the `Series` (a point value, in the above example), and **return** a **transformed version of that value**. `map()` **returns** a new Series where all the values have been transformed by your function.

In [43]:
res = reviews.head(3)
res[['country', 'points']]

,country,points
0,Italy,87
1,Portugal,87
2,US,87


`apply() ` is the equivalent method if we want to transform a whole DataFrame by calling a custom method on each row.

In [58]:
def remean_points(row):
    row.points = row.points - review_points_mean
    return row


res = reviews.apply(remean_points, axis='columns')
res = res.head(3)
res[['country', 'points']]

,country,points
0,Italy,0.57
1,Portugal,0.57
2,US,0.57


If we had called `reviews.apply()` with `axis='index'`, then instead of passing a function to transform each row, we would need to give a function to transform each column.

Pandas will also understand what to do if we perform these operations between Series of equal length. For example, an easy way of combining country and region information in the dataset would be to do the following:

In [6]:
reviews.country + " - " + reviews.region_1

0                      Italy - Etna
1                               NaN
2            US - Willamette Valley
3          US - Lake Michigan Shore
4            US - Willamette Valley
                  ...              
95                France - Juliénas
96                  France - Régnié
97                US - Finger Lakes
98    Italy - Morellino di Scansano
99                 US - Napa Valley
Length: 100, dtype: object

In [7]:
reviews.price - reviews.price.mean()

0           NaN
1    -11.271739
2    -12.271739
3    -13.271739
4     38.728261
        ...    
95    -6.271739
96    -8.271739
97    -6.271739
98     3.728261
99    48.728261
Name: price, Length: 100, dtype: float64

These operators are faster than `map()` or `apply()` because they uses speed ups built into pandas. All of the standard Python operators (`>, <, ==`, and so on) work in this manner.

However, they are not as flexible as `map()` or `apply()`, which can do more advanced things, like applying conditional logic, which cannot be done with addition and subtraction alone.

Example:

There are only so many words you can use when describing a bottle of wine. Is a wine more likely to be "tropical" or "fruity"? Create a Series `descriptor_counts` counting how many times each of these two words appears in the `description` column in the dataset. (For simplicity, let's ignore the capitalized versions of these words.)

In [10]:
n_trop = reviews.description.map(lambda desc: "tropical" in desc).sum()
n_fruity = reviews.description.map(lambda desc: "fruity" in desc).sum()
descriptor_counts = pd.Series([n_trop, n_fruity], index=['tropical', 'fruity'])
descriptor_counts


tropical    4
fruity      8
dtype: int64

We'd like to host these wine reviews on our website, but a rating system ranging from 80 to 100 points is too hard to understand - we'd like to translate them into simple star ratings. A score of 95 or higher counts as 3 stars, a score of at least 85 but less than 95 is 2 stars. Any other score is 1 star.

Also, the Canadian Vintners Association bought a lot of ads on the site, so any wines from Canada should automatically get 3 stars, regardless of points.

Create a series `star_ratings` with the number of stars corresponding to each review in the dataset.

In [20]:
def stars(row):
    if row.country == 'Canada':
        return 3
    elif row.points >= 90:
        return 3
    elif row.points >= 80:
        return 2
    else:
        return 1


star_ratings = reviews.apply(stars, axis='columns')
star_ratings


0     1
1     2
2     2
3     2
4     3
     ..
95    2
96    2
97    2
98    2
99    2
Length: 100, dtype: int64

## Modifying, Adding, Removing, Renaming and Combining Row/Columns

###  Adding Column

In [73]:
df = pd.DataFrame({
    "a": [1, 2, 3, 4],
    "b": ["Bob", "Alice", "Bob", "Alice"],
})
df

,a,b
0,1,Bob
1,2,Alice
2,3,Bob
3,4,Alice


##### direct assignment

In [74]:
df['c'] = [1, 2, 3, 4]
df

,a,b,c
0,1,Bob,1
1,2,Alice,2
2,3,Bob,3
3,4,Alice,4


In [79]:
df['d'] = 1
df

,a,b,c,d
0,1,Bob,1,1
1,2,Alice,2,1
2,3,Bob,3,1
3,4,Alice,4,1


In [81]:
df['d'] = range(0, len(df))
df


,a,b,c,d,e
0,1,Bob,1,0,2
1,2,Alice,2,1,4
2,3,Bob,3,2,6
3,4,Alice,4,3,8


In [82]:
df['e'] = df['a'] + df['c']
df

,a,b,c,d,e
0,1,Bob,1,0,2
1,2,Alice,2,1,4
2,3,Bob,3,2,6
3,4,Alice,4,3,8


In Column "B", Find values, where value="Bob" and replace with "0"

In [127]:
find_Bob_in_b = df['b']=='Bob'
find_Bob_in_b

0     True
1    False
2     True
3    False
Name: b, dtype: bool

In [129]:
df.loc[find_Bob_in_b,'b'] = 'FOUND'
# Not df.loc[find_Bob_in_b] = 'FOUND' => it will replace all Row
df


,a,b,c,d
0,1,FOUND,1,1
1,2,Alice,2,1
2,3,FOUND,3,1
3,4,Alice,4,1


Store the result in a new column

In [130]:
df.loc[find_Bob_in_b,'f'] = 'FOUND'
# Not df.loc[find_Bob_in_b] = 'FOUND' => it will replace all Row
df

,a,b,c,d,f
0,1,FOUND,1,1,FOUND
1,2,Alice,2,1,NaN
2,3,FOUND,3,1,FOUND
3,4,Alice,4,1,NaN


When you add a new colum, it must have the same number of rows. Missing rows are filled with NaN, and extra rows are ignored:

In [74]:
people["pets"] = pd.Series({"bob": 0, "charles": 5, "eugene":1})  # alice is missing, eugene is ignored
people

,weight,birthyear,children,hobby,age,over 30,pets
alice,68,1985,NaN,Biking,36,True,NaN
bob,83,1984,3.0,Dancing,37,True,0.0
charles,112,1992,0.0,NaN,29,False,5.0


##### `insert(position,column,value)`

When adding a new column, it is added at the end (on the right) by default. You can also insert a column anywhere else using the `insert()` method:

In [75]:
people.insert(1, "height", [172, 181, 185])
people

,weight,height,birthyear,children,hobby,age,over 30,pets
alice,68,172,1985,NaN,Biking,36,True,NaN
bob,83,181,1984,3.0,Dancing,37,True,0.0
charles,112,185,1992,0.0,NaN,29,False,5.0


##### `assign()`: Assigning new columns

You can also create new columns by calling the `assign()` method. Note that this returns a new `DataFrame` object, **the original is not modified:**

In [76]:
people.assign(
    body_mass_index = people["weight"] / (people["height"] / 100) ** 2,
    has_pets = people["pets"] > 0
)

,weight,height,birthyear,children,hobby,age,over 30,pets,body_mass_index,has_pets
alice,68,172,1985,NaN,Biking,36,True,NaN,22.985398,False
bob,83,181,1984,3.0,Dancing,37,True,0.0,25.335002,False
charles,112,185,1992,0.0,NaN,29,False,5.0,32.724617,True


In [77]:
people # the original is not modified

,weight,height,birthyear,children,hobby,age,over 30,pets
alice,68,172,1985,NaN,Biking,36,True,NaN
bob,83,181,1984,3.0,Dancing,37,True,0.0
charles,112,185,1992,0.0,NaN,29,False,5.0


Note that you cannot access columns created within the same assignment:

In [78]:
try:
    people.assign(
        body_mass_index = people["weight"] / (people["height"] / 100) ** 2,
        overweight = people["body_mass_index"] > 25 # body_mass_index is not defined at this point
    )
except KeyError as e:
    print("Key error:", e)

Key error: 'body_mass_index'


The solution is to split this assignment in two consecutive assignments:

In [79]:
d6 = people.assign(body_mass_index = people["weight"] / (people["height"] / 100) ** 2)
d6.assign(overweight = d6["body_mass_index"] > 25)

,weight,height,birthyear,children,hobby,age,over 30,pets,body_mass_index,overweight
alice,68,172,1985,NaN,Biking,36,True,NaN,22.985398,False
bob,83,181,1984,3.0,Dancing,37,True,0.0,25.335002,True
charles,112,185,1992,0.0,NaN,29,False,5.0,32.724617,True


Having to create a temporary variable `d6` is not very convenient. You may want to just chain the assigment calls, but it does not work because the `people` object is not actually modified by the first assignment:

In [80]:
try:
    (people
         .assign(body_mass_index = people["weight"] / (people["height"] / 100) ** 2)
         .assign(overweight = people["body_mass_index"] > 25)
    )
except KeyError as e:
    print("Key error:", e)

Key error: 'body_mass_index'


But fear not, there is a simple solution. You can pass a function to the `assign()` method (typically a `lambda` function), and this function will be called with the `DataFrame` as a parameter:

In [81]:
(people
     .assign(body_mass_index = lambda df: df["weight"] / (df["height"] / 100) ** 2)
     .assign(overweight = lambda df: df["body_mass_index"] > 25)
)

,weight,height,birthyear,children,hobby,age,over 30,pets,body_mass_index,overweight
alice,68,172,1985,NaN,Biking,36,True,NaN,22.985398,False
bob,83,181,1984,3.0,Dancing,37,True,0.0,25.335002,True
charles,112,185,1992,0.0,NaN,29,False,5.0,32.724617,True


Problem solved!

### Adding Row

In [39]:
arr= np.random.randint(10, 100, size=(6,4))
df = pd.DataFrame(data=arr,columns=["a", "b", "c", "d"])
df.index = "p q r s t u".split()

In [42]:
print(df.loc['p'])
print(df.iloc[0])

a    81
b    86
c    65
d    23
Name: p, dtype: int32
a    81
b    86
c    65
d    23
Name: p, dtype: int32


In [44]:
df.loc['x'] = [1,2,3,4]
df.tail()

,a,b,c,d
r,79,12,25,51
s,10,99,54,73
t,98,55,14,90
u,61,62,63,55
x,1,2,3,4


### Removing Rows/Columns

- `drop` method:
  - `drop(columns,axis=1)`
  - `drop(index,axis=0)`
  - `drop(labels,axis=1)`
  - `drop(labels,axis=0)`
-  `pop`

In [18]:
arr = np.random.randint(10, 100, size=(4,8))
df = pd.DataFrame(data=arr,columns=["a", "b", "c", "d", "e", "f", "g", "h"])
df['a+b'] = df['a'] + df['b']
df['a-b'] = df['a'] * df['b']
df

,a,b,c,d,e,f,g,h,a+b,a-b
0,53,98,47,62,69,88,48,22,151,5194
1,58,11,46,71,46,26,61,49,69,638
2,85,97,64,76,38,56,67,94,182,8245
3,53,51,29,23,60,36,46,28,104,2703


In [19]:
delC = df.pop('c')  # removes column c
del df["d"] # removes column d
df

,a,b,e,f,g,h,a+b,a-b
0,53,98,69,88,48,22,151,5194
1,58,11,46,26,61,49,69,638
2,85,97,38,56,67,94,182,8245
3,53,51,60,36,46,28,104,2703


In [20]:
df.drop(columns=['e','f','a-b']) # removes columns e, f, a-b

,a,b,g,h,a+b
0,53,98,48,22,151
1,58,11,61,49,69
2,85,97,67,94,182
3,53,51,46,28,104


In [21]:
df

,a,b,e,f,g,h,a+b,a-b
0,53,98,69,88,48,22,151,5194
1,58,11,46,26,61,49,69,638
2,85,97,38,56,67,94,182,8245
3,53,51,60,36,46,28,104,2703


In [22]:
df.drop(columns=['e', 'f', 'a-b'], inplace=True)  # original df is modified

In [23]:
df

,a,b,g,h,a+b
0,53,98,48,22,151
1,58,11,61,49,69
2,85,97,67,94,182
3,53,51,46,28,104


In [24]:
df.drop(1,inplace=True) # removes row 1
df

,a,b,g,h,a+b
0,53,98,48,22,151
2,85,97,67,94,182
3,53,51,46,28,104


In [25]:
df.index

Int64Index([0, 2, 3], dtype='int64')

In [28]:
df.drop(df.index[[0,2]])


,a,b,g,h,a+b
2,85,97,67,94,182


### Renaming

### Combining


## Data Types and Missing Values

In [67]:
reviews = pd.read_csv('winemag-data-130k-v2-mod.csv', index_col=0)
reviews.head(n=2)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,77,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


### `dtypes`, `astype()`

The data type for a column in a DataFrame or a Series is known as the dtype.

In [55]:
reviews.dtypes


country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

You can use the dtype property to grab the type of a specific column. For instance, we can get the dtype of the price column in the reviews DataFrame:



In [53]:
reviews.price.dtype

dtype('float64')

It's possible to convert a column of one type into another wherever such a conversion makes sense by using the `astype()` function. For example, we may transform the points column from its existing int64 data type into a float64 data type:



In [56]:
reviews.points.astype('float64')

0     77.0
1     87.0
2     87.0
3     87.0
4     90.0
      ... 
95    88.0
96    88.0
97    88.0
98    88.0
99    88.0
Name: points, Length: 100, dtype: float64

### Missing data


Entries missing values are given the value `NaN`, short for `"Not a Number"`. For technical reasons these NaN values are always of the float64 dtype.

In [71]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id':np.random.randint(1000, 2000, size=5),
	'marks':np.random.randint(50,100,size=5)
}
df = pd.DataFrame(data)
df

,roll_no,ppr_id,marks
0,37,1690,56
1,15,1700,87
2,53,1364,55
3,94,1372,90
4,44,1291,76


In [72]:
nan_idx = np.random.randint(0,5,3)
df['marks'][nan_idx] = np.nan
# df['marks'][[1,4,3]] = np.nan
df

,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,NaN
2,53,1364,55.0
3,94,1372,NaN
4,44,1291,NaN


#### `isnull()` and `notnull()`


Pandas provides some methods specific to missing data. To select NaN entries you can use `pd.isnull()` (or its companion `pd.notnull()`). This is meant to be used thusly:


In [73]:
df.isnull()

,roll_no,ppr_id,marks
0,False,False,False
1,False,False,True
2,False,False,False
3,False,False,True
4,False,False,True


In [75]:
df.isnull().sum()

roll_no    0
ppr_id     0
marks      3
dtype: int64

How many reviews in the dataset are missing a price?

In [92]:
missing_price_reviews = reviews[reviews.price.isnull()]
n_missing_prices = len(missing_price_reviews)
# Cute alternative solution: if we sum a boolean series, True is treated as 1 and False as 0
n_missing_prices = reviews.price.isnull().sum()
# or equivalently:
n_missing_prices = pd.isnull(reviews.price).sum()
n_missing_prices

8

In [74]:
df['marks'].isnull()

0    False
1     True
2    False
3     True
4     True
Name: marks, dtype: bool

#### `fillna`

In [77]:
df.fillna("FILLED")

,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,FILLED
2,53,1364,55.0
3,94,1372,FILLED
4,44,1291,FILLED


In [78]:
df

,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,NaN
2,53,1364,55.0
3,94,1372,NaN
4,44,1291,NaN


In [80]:
df = df.fillna("FILLED")
df.fillna("FILLED",inplace=True)
df


,roll_no,ppr_id,marks
0,37,1690,56.0
1,15,1700,FILLED
2,53,1364,55.0
3,94,1372,FILLED
4,44,1291,FILLED


In [82]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id': np.random.randint(1000, 2000, size=5),
	'marks': np.random.randint(50, 100, size=5)
}
df = pd.DataFrame(data)
nan_idx = np.random.randint(0, 5, 3)
df['marks'][nan_idx] = np.nan
df

,roll_no,ppr_id,marks
0,60,1764,53.0
1,58,1443,NaN
2,23,1603,96.0
3,18,1626,NaN
4,21,1656,NaN


In [84]:
df.marks.fillna(df.marks.mean(), inplace=True)

In [85]:
df

,roll_no,ppr_id,marks
0,60,1764,53.0
1,58,1443,74.5
2,23,1603,96.0
3,18,1626,74.5
4,21,1656,74.5


What are the most common wine-producing regions? Create a Series counting the number of times each value occurs in the `region_1` field. This field is often missing data, so replace missing values with `Unknown`. Sort in descending order.  Your output should look something like this:

In [93]:
reviews_per_region = reviews.region_1.fillna(
    'Unknown').value_counts().sort_values(ascending=False)
reviews_per_region

Unknown                       12
Sicilia                       11
Napa Valley                    7
Columbia Valley (WA)           5
Virginia                       3
Alsace                         3
Willamette Valley              3
Alexander Valley               2
Etna                           2
Paso Robles                    2
Terre Siciliane                2
Champagne                      2
Sonoma Coast                   2
Aglianico del Vulture          1
Sonoma County                  1
Puglia                         1
Ancient Lakes                  1
Chablis                        1
Central Coast                  1
Vin de France                  1
Ribera del Duero               1
Lake County                    1
Vernaccia di San Gimignano     1
Dry Creek Valley               1
Beaujolais-Villages            1
Navarra                        1
McLaren Vale                   1
South Australia                1
Clarksburg                     1
Brouilly                       1
California

#### `dropna`

In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id': np.random.randint(1000, 2000, size=5),
	'marks': np.random.randint(50, 100, size=5)
}
df = pd.DataFrame(data)
df['marks'][[0, 2, 4]] = np.nan
df['roll_no'][[0, 2]] = np.nan
df


,roll_no,ppr_id,marks
0,NaN,1074,NaN
1,4.0,1867,60.0
2,NaN,1103,NaN
3,90.0,1699,54.0
4,70.0,1372,NaN


In [99]:
df.dropna()

,roll_no,ppr_id,marks
1,4.0,1867,60.0
3,90.0,1699,54.0


## Saving & loading files

Pandas can save `DataFrame`s to various backends, including file formats such as CSV, Excel, JSON, HTML and HDF5, or to a SQL database. Let's create a `DataFrame` to demonstrate this:

In [154]:
df = pd.DataFrame({
	"id":np.arange(10),
	'b':np.random.normal(size=10),
	"c":pd.Series(np.random.choice(["cat",'dog',"hippo"],replace=True,size=10))
})
df.head()

,id,b,c
0,0,0.212280,cat
1,1,0.492354,hippo
2,2,1.667453,dog
3,3,-1.904760,cat
4,4,-0.520301,hippo


### Saving
Let's save it to CSV, HTML and JSON:

In [150]:
df.to_csv("my_df.csv")
df.to_csv("my_df_index_false.csv", index=False)
df.to_html("my_df.html")
df.to_json("my_df.json")


### Loading

In [151]:
import os
print(os.getcwd())
print(os.listdir())

d:\CSE\Others\ML-py\01pandas
['img', 'iris.csv', 'my_df.csv', 'my_df.html', 'my_df.json', 'my_df_index_false.csv', 'pandas.ipynb', 'README.md']


Now let's load our CSV file back into a `DataFrame`:

- Loading from file saved without `index=False`

In [153]:
my_df_loaded = pd.read_csv("my_df.csv")
my_df_loaded.head()

,Unnamed: 0,id,b,c
0,0,0,1.106266,hippo
1,1,1,-1.612778,cat
2,2,2,-0.264879,cat
3,3,3,-0.213137,cat
4,4,4,-0.184308,hippo


- Loading from file saved with `index=False`

In [156]:
my_df_loaded_index_false = pd.read_csv("my_df_index_false.csv")
my_df_loaded_index_false.head()

,id,b,c
0,0,1.106266,hippo
1,1,-1.612778,cat
2,2,-0.264879,cat
3,3,-0.213137,cat
4,4,-0.184308,hippo


- Loading from file saved without `index=False`, without `Unnamed: 0` column

The `pd.read_csv()` function is well-endowed, with over 30 optional parameters you can specify. For example, you can see in this dataset that the CSV file has a built-in index, which pandas did not pick up on automatically. To make pandas use that column for the index (instead of creating a new one from scratch), we can specify an `index_col`.

In [158]:
my_df_loaded = pd.read_csv("my_df.csv",index_col=0)
my_df_loaded.head()

,id,b,c
0,0,1.106266,hippo
1,1,-1.612778,cat
2,2,-0.264879,cat
3,3,-0.213137,cat
4,4,-0.184308,hippo


- Or Dropping "Unnamed: 0" Column

In [159]:
my_df_loaded = pd.read_csv("my_df.csv")
my_df_loaded = my_df_loaded.drop(columns=['Unnamed: 0'])
my_df_loaded.head()


,id,b,c
0,0,1.106266,hippo
1,1,-1.612778,cat
2,2,-0.264879,cat
3,3,-0.213137,cat
4,4,-0.184308,hippo


As you might guess, there are similar `read_json`, `read_html`, `read_excel` functions as well.  We can also read data straight from the Internet. For example, let's load the top 1,000 U.S. cities from github:

In [64]:
us_cities = None
try:
    csv_url = "https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv"
    us_cities = pd.read_csv(csv_url, index_col=0)
    us_cities = us_cities.head()
except IOError as e:
    print(e)
us_cities

,State,Population,lat,lon
City,,,,
Marysville,Washington,63269,48.051764,-122.177082
Perris,California,72326,33.782519,-117.228648
Cleveland,Ohio,390113,41.499320,-81.694361
Worcester,Massachusetts,182544,42.262593,-71.802293
Columbia,South Carolina,133358,34.000710,-81.034814


### Minimize the size of Large DataSet

[wine-reviews-dataset](https://www.kaggle.com/zynicide/wine-reviews)

In [21]:
data = pd.read_csv('winemag-data-130k-v2.csv')
print(f"Pre Shape : {data.shape}")
# read only first 100 rows
data = pd.read_csv('winemag-data-130k-v2.csv', nrows=100, index_col=0)
data.head(n=2)

Pre Shape : (129971, 14)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [22]:
# Save the dataframe to a csv file
data.to_csv("winemag-data-130k-v2-mod.csv")

In [24]:

new_data = pd.read_csv('winemag-data-130k-v2-mod.csv', index_col=0)
print(f"Post Shape: {new_data.shape}")
new_data.head(n=2)

Post Shape: (100, 13)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


## Operations on `DataFrame`s

Although `DataFrame`s do not try to mimick NumPy arrays, there are a few similarities. Let's create a `DataFrame` to demonstrate this:

In [101]:
data = {
	'roll_no': np.random.randint(1, 100, size=5),
	'ppr_id': np.random.randint(1000, 2000, size=5),
	'marks': np.random.randint(50, 100, size=5)
}
df = pd.DataFrame(data)
df


,roll_no,ppr_id,marks
0,83,1930,83
1,45,1954,57
2,92,1638,67
3,92,1126,91
4,56,1369,96


In [114]:
df['marks'].sum()

394

In [115]:
df['marks'].mean()

78.8

In [116]:
df['marks'].cumsum()

0     83
1    140
2    207
3    298
4    394
Name: marks, dtype: int32

In [117]:
df['roll_no'].count()

5

In [118]:
df['marks'].min()

57

In [119]:
df['marks'].max()

96

In [120]:
df['marks'].var()

269.2

In [121]:
df['marks'].std()

16.407315441594946

In [122]:
df.corr()

,roll_no,ppr_id,marks
roll_no,1.000000,-0.352082,0.25746
ppr_id,-0.352082,1.000000,-0.70311
marks,0.257460,-0.703110,1.00000


Similarly, adding a single value to a `DataFrame` will add that value to all elements in the `DataFrame`. This is called *broadcasting*:

In [102]:
df + 1

,roll_no,ppr_id,marks
0,84,1931,84
1,46,1955,58
2,93,1639,68
3,93,1127,92
4,57,1370,97


Of course, the same is true for all other binary operations, including arithmetic (`*`,`/`,`**`...) and conditional (`>`, `==`...) operations:

In [104]:
df >= 500

,roll_no,ppr_id,marks
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False


Aggregation operations, such as computing the `max`, the `sum` or the `mean` of a `DataFrame`, apply to each column, and you get back a `Series` object:

In [105]:
df.mean()

roll_no      73.6
ppr_id     1603.4
marks        78.8
dtype: float64

The `all` method is also an aggregation operation: it checks whether all values are `True` or not. Let's see during which months all students got a grade greater than `5`:

In [106]:
(df > 50).all()

roll_no    False
ppr_id      True
marks       True
dtype: bool

Most of these functions take an optional `axis` parameter which lets you specify along which axis of the `DataFrame` you want the operation executed. The default is `axis=0`, meaning that the operation is executed vertically (on each column). You can set `axis=1` to execute the operation horizontally (on each row). For example, let's find out which students had all grades greater than `5`:

In [108]:
(df > 50).all(axis = 1)

0     True
1    False
2     True
3     True
4     True
dtype: bool

The `any` method returns `True` if any value is True. Let's see who got at least one grade 10:

In [110]:
(df == 92).any(axis = 1)

0    False
1    False
2     True
3     True
4    False
dtype: bool

If you add a `Series` object to a `DataFrame` (or execute any other binary operation), pandas attempts to broadcast the operation to all *rows* in the `DataFrame`. This only works if the `Series` has the same size as the `DataFrame`s rows. For example, let's subtract the `mean` of the `DataFrame` (a `Series` object) from the `DataFrame`:

In [111]:
df - df.mean()

,roll_no,ppr_id,marks
0,9.4,326.6,4.2
1,-28.6,350.6,-21.8
2,18.4,34.6,-11.8
3,18.4,-477.4,12.2
4,-17.6,-234.4,17.2


If you want to subtract the global mean from every grade, here is one way to do it:

In [112]:
df - df.values.mean() # subtracts the global mean 

,roll_no,ppr_id,marks
0,-502.266667,1344.733333,-502.266667
1,-540.266667,1368.733333,-528.266667
2,-493.266667,1052.733333,-518.266667
3,-493.266667,540.733333,-494.266667
4,-529.266667,783.733333,-489.266667


## Grouping and Sorting


### Groupwise analysis


One function we've been using heavily thus far is the `value_counts()` function. We can replicate what `value_counts()` does by doing the following:

In [21]:
reviews.groupby('points').points.count()

points
50     1
66     1
70     2
77     1
80     1
85     9
86    49
87    23
88    11
90     1
99     1
Name: points, dtype: int64

`groupby()` created a group of reviews which allotted the same point values to the given wines. Then, for each of these groups, we grabbed the `points() `column and counted how many times it appeared. `value_counts()` is just a shortcut to this `groupby()` operation.



We can use any of the**summary functions** we've used before with this data. For example, to get the cheapest wine in each point value category, we can do the following:

In [22]:
reviews.groupby('points').price.min()

points
50    28.0
66    13.0
70    14.0
77     NaN
80    30.0
85    10.0
86     9.0
87    12.0
88    12.0
90    65.0
99     NaN
Name: price, dtype: float64

You can think of each group we generate as being a slice of our DataFrame containing only data with values that match. This DataFrame is accessible to us directly using the `apply()` method, and we can then manipulate the data in any way we see fit. For example, here's one way of selecting the name of the first wine reviewed from each winery in the dataset:

In [23]:
reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

winery
Acrobat                                             Acrobat 2013 Pinot Noir (Oregon)
Adega Cooperativa do Cartaxo       Adega Cooperativa do Cartaxo 2014 Bridão Touri...
Aresti                             Aresti 2014 Special Release Reserva Carmenère ...
Baglio di Pianetto                 Baglio di Pianetto 2007 Ficiligno White (Sicilia)
Basel Cellars                      Basel Cellars 2013 Inspired Red (Columbia Vall...
                                                         ...                        
Vignerons de Bel Air                Vignerons de Bel Air 2011 Eté Indien  (Brouilly)
Vignerons des Terres Secrètes      Vignerons des Terres Secrètes 2015  Mâcon-Mill...
Viticultori Associati Canicatti    Viticultori Associati Canicatti 2008 Scialo Re...
Yalumba                            Yalumba 2016 Made With Organic Grapes Chardonn...
Z'IVO                               Z'IVO 2015 Rosé of Pinot Noir (Eola-Amity Hills)
Length: 91, dtype: object

For even more fine-grained control, you can also group by more than one column. For an example, here's how we would pick out the best wine by country and province:

In [26]:
reviews.groupby(['country', 'province']).apply(
    lambda df: df.loc[df.points.idxmax()]).head(3)

country  \
country   province                      
Argentina Mendoza Province  Argentina   
          Other             Argentina   
Australia South Australia   Australia   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  Raw black-cherry aromas are direct and simple ...   
          Other             Baked plum, molasses, balsamic vinegar and che...   
Australia South Australia   This medium-bodied Chardonnay features aromas ...   

                                         designation  points  price  \
country   province                                                    
Argentina Mendoza Province       Winemaker Selection      87   13.0   
          Other                                Felix      87   30.0   
Australia South Australia   Made With Organic Grapes      86   18.0   

                                    province         region_1 region_2  \
country   province                                                       
Argentina Mendoza Province  Mendoza Province          Mendoza      NaN   
          Other                        Other         Cafayate      NaN   
Australia South Australia    South Australia  South Australia      NaN   

                                  taster_name taster_twitter_handle  \
country   province                                                    
Argentina Mendoza Province  Michael Schachner           @wineschach   
          Other             Michael Schachner           @wineschach   
Australia South Australia      Joe Czerwinski                @JoeCz   

                                                                        title  \
country   province                                                              
Argentina Mendoza Province  Gaucho Andino 2011 Winemaker Selection Malbec ...   
          Other                    Felix Lavaque 2010 Felix Malbec (Cafayate)   
Australia South Australia   Yalumba 2016 Made With Organic Grapes Chardonn...   

                               variety         winery  
country   province                                     
Argentina Mendoza Province      Malbec  Gaucho Andino  
          Other                 Malbec  Felix Lavaque  
Australia South Australia   Chardonnay        Yalumba

Another `groupby()` method worth mentioning is `agg()`, which lets you run a bunch of different functions on your DataFrame simultaneously. For example, we can generate a simple statistical summary of the dataset as follows:

In [27]:
reviews.groupby(['country']).price.agg([len, min, max])

,len,min,max
country,,,
Argentina,2.0,13.0,30.0
Australia,2.0,18.0,20.0
Austria,1.0,12.0,12.0
Chile,5.0,9.0,22.0
France,14.0,9.0,58.0
Germany,4.0,9.0,24.0
Italy,24.0,10.0,35.0
Portugal,2.0,15.0,15.0
Spain,3.0,15.0,28.0


### Sorting

You can sort a `DataFrame` by calling its `sort_index` method. By default it sorts the rows by their **index label**, in ascending order, but let's reverse the order:

In [37]:
people_dict = {
    "country": pd.Series(['BD', 'IN', 'PAK', 'SL', 'US', 'IN']),
   	"name": pd.Series(['A', 'B', 'C', 'D', 'E', 'F']),
	   "cgpa":pd.Series([3.56, 4.00, 3.55, 3.86, 3.99, 3.89])
}
people = pd.DataFrame(people_dict)
people

,country,name,cgpa
0,BD,A,3.56
1,IN,B,4.00
2,PAK,C,3.55
3,SL,D,3.86
4,US,E,3.99
5,IN,F,3.89


In [38]:
people.sort_index(ascending=False).head(n=3)

,country,name,cgpa
5,IN,F,3.89
4,US,E,3.99
3,SL,D,3.86


Note that `sort_index` returned a sorted *copy* of the `DataFrame`. To modify `people` directly, we can set the `inplace` argument to `True`. Also, we can sort the columns instead of the rows by setting `axis=1`:

In [39]:
people.sort_index(axis=1,ascending=False, inplace=True)
people

,name,country,cgpa
0,A,BD,3.56
1,B,IN,4.00
2,C,PAK,3.55
3,D,SL,3.86
4,E,US,3.99
5,F,IN,3.89


To sort the `DataFrame` by the values instead of the labels, we can use `sort_values` and specify the column to sort by:

In [41]:
people.sort_values(by=["name"], ascending=False,inplace=True)
people

,name,country,cgpa
5,F,IN,3.89
4,E,US,3.99
3,D,SL,3.86
2,C,PAK,3.55
1,B,IN,4.00
0,A,BD,3.56


In [46]:
people.sort_values(by=["cgpa", "name"])

,name,country,cgpa
2,C,PAK,3.55
0,A,BD,3.56
3,D,SL,3.86
5,F,IN,3.89
4,E,US,3.99
1,B,IN,4.00


### More example:

In [28]:
reviews_written = reviews.groupby('taster_twitter_handle').size()
reviews_written

taster_twitter_handle
@AnneInVino          1
@JoeCz               2
@gordone_cellars     1
@kerinokeefe        13
@mattkettmann        3
@paulgwine           6
@vboone             16
@vossroger          16
@wawinereport        6
@wineschach         10
dtype: int64

What is the **best** wine I can buy for a given amount of money? Create a `Series` whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that `4.0` dollars is at the top and `3300.0` dollars is at the bottom).

In [31]:
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()
best_rating_per_price

price
9.0      86
10.0     86
11.0     86
12.0     88
13.0     87
14.0     87
15.0     87
16.0     87
17.0     87
18.0     88
19.0     88
20.0     88
21.0     86
22.0     88
23.0     88
24.0     87
25.0     86
26.0     86
27.0     87
28.0     50
29.0     86
30.0     88
32.0     87
34.0     87
35.0     87
40.0     86
46.0     86
50.0     86
55.0     88
58.0     86
65.0     90
69.0     87
75.0     88
100.0    86
Name: points, dtype: int64

What are the minimum and maximum prices for each `variety` of wine? Create a `DataFrame` whose index is the `variety` category from the dataset and whose values are the `min` and `max` values thereof.

In [41]:
price_extremes = reviews.groupby('variety').price.agg([min, max])
price_extremes[:3]


,min,max
variety,,
Aglianico,32.0,32.0
Albariño,16.0,20.0
Bordeaux-style Red Blend,46.0,75.0


Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [48]:
reviewer_mean_ratings=reviews.groupby('taster_name').points.mean()
reviewer_mean_ratings

taster_name
Alexander Peartree    87.000000
Anna Lee C. Iijima    86.800000
Anne Krebiehl MW      88.000000
Jim Gordon            86.000000
Joe Czerwinski        86.000000
Kerin O’Keefe         86.000000
Matt Kettmann         86.666667
Michael Schachner     80.800000
Paul Gregutt          87.000000
Roger Voss            86.812500
Sean P. Sullivan      86.333333
Virginie Boone        86.625000
Name: points, dtype: float64

In [49]:
reviewer_mean_ratings.describe()

count    12.000000
mean     86.169792
std       1.782245
min      80.800000
25%      86.000000
50%      86.645833
75%      86.859375
max      88.000000
Name: points, dtype: float64

What combination of countries and varieties are most common? Create a `Series` whose index is a `MultiIndex`of `{country, variety}` pairs. For example, a pinot noir produced in the US should map to `{"US", "Pinot Noir"}`. Sort the values in the `Series` in descending order based on wine count.

In [51]:
country_variety_counts = reviews.groupby(
    ['country', 'variety']).size().sort_values(ascending=False)
country_variety_counts


country    variety                   
US         Pinot Noir                    6
           Red Blend                     5
           Cabernet Sauvignon            5
France     Gamay                         5
Italy      Red Blend                     4
US         Sauvignon Blanc               4
           Chardonnay                    4
Italy      White Blend                   4
US         Riesling                      3
Italy      Nero d'Avola                  3
US         Bordeaux-style Red Blend      3
Germany    Riesling                      3
Argentina  Malbec                        2
US         Meritage                      2
Italy      Sangiovese                    2
France     Gewürztraminer                2
           Chardonnay                    2
           Champagne Blend               2
US         Merlot                        2
           Pinot Gris                    2
           Chenin Blanc                  1
           Cabernet Franc                1
           Petit

## Handling String Data - Converting Reality to Numbers

In [45]:
people_dict = {
	# panda series from a list of A to D
	"name":pd.Series(['A','B','C','D']),
	"gender":pd.Series(['Female','Male','Female','Female'])
}
people = pd.DataFrame(people_dict)
people

,name,gender
0,A,Female
1,B,Male
2,C,Female
3,D,Female


In [47]:
def f(g):
	if g == 'Male':
		return 0
	else:
		return 1


people['sex'] = people.gender.apply(f)
people

,name,gender,sex
0,A,Female,1
1,B,Male,0
2,C,Female,1
3,D,Female,1
